In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback
import xgboost as xgb

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 500 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 
            
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)

In [3]:
column_usefulness = [('bc_price', 16597), ('manager_worths', 14684), ('price_by_NTACode', 13343), ('latitude', 12299), ('longitude', 12051), ('desc_words_length', 11477), ('manager_id', 11136), ('price_by_address1', 11125), ('Yday', 10884), ('address1', 10458), ('building_worths', 10298), ('building_id', 10142), ('desc_letters_count', 9832), ('real_minus_expected_price', 9815), ('price_by_area', 9742), ('real_over_expected_price', 9374), ('Day', 9066), ('worth_by_address1', 8782), ('hour', 7722), ('price_per_bathroom', 7590), ('desc_words_count', 7481), ('price_per_bed_and_bath', 7387), ('worth_by_NTACode', 6993), ('photos_count', 6730), ('NTACode', 6324), ('price_by_BoroCode', 6098), ('features_count', 5668), ('Wday', 4445), ('price_per_bedroom', 3903), ('sparse_nofee', 3056), ('bedrooms', 2833), ('sparse_furnished', 1865), ('bathrooms', 1702), ('sparse_laundryinbuilding', 1425), ('sparse_hardwoodfloors', 1132), ('sparse_laundryinunit', 1088), ('Month', 1006), ('sparse_exclusive', 771), ('sparse_dogsallowed', 770), ('BoroCode', 749), ('sparse_catsallowed', 718), ('sparse_prewar', 663), ('sparse_privateoutdoorspace', 634), ('sparse_reducedfee', 631), ('avenue', 595), ('street', 569), ('other_address', 553), ('sparse_commonoutdoorspace', 545), ('Zero_building_id', 534), ('sparse_elevator', 533), ('east', 524), ('sparse_doorman', 510), ('sparse_parkingspace', 506), ('sparse_dishwasher', 500), ('sparse_highspeedinternet', 476), ('sparse_terrace', 424), ('sparse_fitnesscenter', 423), ('sparse_loft', 413), ('sparse_outdoorspace', 406), ('sparse_actualapt', 399), ('sparse_roofdeck', 359), ('sparse_balcony', 346), ('sparse_diningroom', 318), ('sparse_simplex', 292), ('sparse_swimmingpool', 283), ('sparse_shorttermallowed', 263), ('sparse_wheelchairaccess', 248), ('west', 245), ('sparse_newconstruction', 238), ('sparse_garden', 231), ('sparse_multilevel', 201), ('sparse_patio', 199), ('sparse_hardwood', 198), ('worth_by_BoroCode', 179), ('sparse_fireplace', 168), ('sparse_highceiling', 159), ('sparse_petsok', 154), ('sparse_dryerinunit', 153), ('sparse_stainlesssteelappliances', 148), ('sparse_garage', 119), ('south', 118), ('sparse_storage', 118), ('sparse_outdoorareas', 117), ('sparse_lndrybldg', 113), ('sparse_liveinsuper', 110), ('sparse_new', 103), ('sparse_renovated', 99), ('sparse_photos', 88), ('sparse_highceilings', 79), ('sparse_onsitelaundry', 79), ('sparse_view', 78), ('sparse_flex3', 67), ('sparse_centrala', 66), ('sparse_concierge', 64), ('sparse_residentslounge', 63), ('sparse_assignedparkingspace', 62), ('sparse_allutilitiesincluded', 58), ('sparse_onsitegarage', 53), ('north', 46), ('sparse_lowrise', 44), ('sparse_washerinunit', 44)]
leaky = ['manager_worths', 'building_worths', 'price_by_area', 'real_minus_expected_price', 'real_over_expected_price', 'price_by_address1', 'worth_by_address1', 'price_by_BoroCode', 'worth_by_BoroCode', 'price_by_NTACode', 'worth_by_NTACode']
all_columns = ['BoroCode', 'Day', 'Month', 'NTACode', 'Wday', 'Yday', 'Zero_building_id', 'Zero_manager_id', 'address1', 'avenue', 'bathrooms', 'bc_price', 'bedrooms', 'building_id', 'building_worths', 'desc_letters_count', 'desc_words_count', 'desc_words_length', 'east', 'features_count', 'hour', 'latitude', 'longitude', 'manager_id', 'manager_worths', 'north', 'other_address', 'photos_count', 'price_by_BoroCode', 'price_by_NTACode', 'price_by_address1', 'price_by_area', 'price_per_bathroom', 'price_per_bed_and_bath', 'price_per_bedroom', 'real_minus_expected_price', 'real_over_expected_price', 'south', 'sparse_24', 'sparse_2fullbaths', 'sparse_actualapt', 'sparse_airconditioning', 'sparse_allutilitiesincluded', 'sparse_assignedparkingspace', 'sparse_attendedlobby', 'sparse_backyard', 'sparse_balconies', 'sparse_balcony', 'sparse_basementstorage', 'sparse_bikeroom', 'sparse_bikestorage', 'sparse_billiardsroom', 'sparse_brownstone', 'sparse_buildingcommonoutdoorspace', 'sparse_businesscenter', 'sparse_cable', 'sparse_catsallowed', 'sparse_centrala', 'sparse_centralac', 'sparse_centralair', 'sparse_chef', 'sparse_chefskitchen', 'sparse_children', 'sparse_childrensplayroom', 'sparse_cityview', 'sparse_commonbackyard', 'sparse_commongarden', 'sparse_commonoutdoorspace', 'sparse_commonparking', 'sparse_commonroofdeck', 'sparse_commonterrace', 'sparse_communityrecreationfacilities', 'sparse_concierge', 'sparse_conciergeservice', 'sparse_condofinishes', 'sparse_cook', 'sparse_courtyard', 'sparse_deck', 'sparse_decorativefireplace', 'sparse_diningroom', 'sparse_dishwasher', 'sparse_dogsallowed', 'sparse_doorman', 'sparse_drycleaningservice', 'sparse_dryer', 'sparse_dryerinbuilding', 'sparse_dryerinunit', 'sparse_duplex', 'sparse_eatinkitchen', 'sparse_elev', 'sparse_elevator', 'sparse_elevbldg', 'sparse_exclusive', 'sparse_exposedbrick', 'sparse_fireplace', 'sparse_fireplaces', 'sparse_fitness', 'sparse_fitnesscenter', 'sparse_flex2', 'sparse_flex3', 'sparse_ftdoorman', 'sparse_fullservicegarage', 'sparse_fulltimedoorman', 'sparse_furnished', 'sparse_garage', 'sparse_garden', 'sparse_gourmetkitchen', 'sparse_granitecountertops', 'sparse_granitekitchen', 'sparse_greenbuilding', 'sparse_guarantorsaccepted', 'sparse_gutrenovated', 'sparse_gym', 'sparse_gyminbuilding', 'sparse_hardwood', 'sparse_hardwoodfloors', 'sparse_healthclub', 'sparse_highceiling', 'sparse_highceilings', 'sparse_highrise', 'sparse_highspeedinternet', 'sparse_hirise', 'sparse_housekeeping', 'sparse_indoorpool', 'sparse_inunitwasher', 'sparse_largelivingroom', 'sparse_laundry', 'sparse_laundryinbuilding', 'sparse_laundryinunit', 'sparse_laundryonfloor', 'sparse_laundryroom', 'sparse_light', 'sparse_live', 'sparse_liveinsuper', 'sparse_liveinsuperintendent', 'sparse_lndrybldg', 'sparse_loft', 'sparse_lounge', 'sparse_loungeroom', 'sparse_lowrise', 'sparse_luxurybuilding', 'sparse_mailroom', 'sparse_marblebath', 'sparse_marblebathroom', 'sparse_microwave', 'sparse_midrise', 'sparse_mrcleanapproved', 'sparse_multilevel', 'sparse_new', 'sparse_newconstruction', 'sparse_newlyrenovated', 'sparse_nofee', 'sparse_nopets', 'sparse_nursery', 'sparse_onsitegarage', 'sparse_onsitelaundry', 'sparse_onsiteparking', 'sparse_onsiteparkingavailable', 'sparse_onsiteparkinglot', 'sparse_onsitesuper', 'sparse_ornateprewardetails', 'sparse_outdoorareas', 'sparse_outdoorentertainmentspace', 'sparse_outdoorpool', 'sparse_outdoorspace', 'sparse_packageroom', 'sparse_parking', 'sparse_parkingspace', 'sparse_patio', 'sparse_penthouse', 'sparse_petfriendly', 'sparse_pets', 'sparse_petsallowed', 'sparse_petsok', 'sparse_petsonapproval', 'sparse_photos', 'sparse_playroom', 'sparse_pool', 'sparse_postwar', 'sparse_prewar', 'sparse_privatebackyard', 'sparse_privatebalcony', 'sparse_privatedeck', 'sparse_privatelaundryroomoneveryfloor', 'sparse_privateoutdoorspace', 'sparse_privateparking', 'sparse_privateroofdeck', 'sparse_privateterrace', 'sparse_publicoutdoor', 'sparse_reducedfee', 'sparse_renovated', 'sparse_residentsgarden', 'sparse_residentslounge', 'sparse_roofaccess', 'sparse_roofdeck', 'sparse_rooftopdeck', 'sparse_rooftopterrace', 'sparse_roomyclosets', 'sparse_satellitetv', 'sparse_sauna', 'sparse_screeningroom', 'sparse_sharedbackyard', 'sparse_sharesok', 'sparse_shorttermallowed', 'sparse_simplex', 'sparse_skitchen', 'sparse_skylight', 'sparse_splayroom', 'sparse_sskitchen', 'sparse_stainlesssteelappliances', 'sparse_stepstothepark', 'sparse_storage', 'sparse_storagefacilitiesavailable', 'sparse_storageroom', 'sparse_sublet', 'sparse_subway', 'sparse_sundeck', 'sparse_swimmingpool', 'sparse_tenantlounge', 'sparse_terrace', 'sparse_terraces', 'sparse_tonsofnaturallight', 'sparse_valet', 'sparse_valetparking', 'sparse_valetservices', 'sparse_valetservicesincludingdrycleaning', 'sparse_videointercom', 'sparse_view', 'sparse_virtualdoorman', 'sparse_walkincloset', 'sparse_wallsofwindows', 'sparse_washer', 'sparse_washerinunit', 'sparse_wheelchairaccess', 'sparse_wheelchairramp', 'sparse_wifi', 'sparse_wifiaccess', 'sparse_work', 'street', 'west', 'worth_by_BoroCode', 'worth_by_NTACode', 'worth_by_address1']
sparse_cols = [x[0] for x in column_usefulness if x[0].startswith('sparse')][:15]
non_sparse_cols = [x[0] for x in column_usefulness if not x[0].startswith('sparse')]

# Add a bunch of features to train_X

In [4]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric

In [5]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)
train_X_with_layer_1 = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json')

In [ ]:
normalize_high_cordiality_data()

In [176]:
#%%time
#PCA
'''
n_components = 3
pca_columns = ['layer1_pca_{}'.format(i) for i in range(n_components)]
for column in pca_columns:
    train_X_with_layer_1[column] = 0
pca = PCA(n_components=n_components)
pca_representation = pca.fit_transform(train_X_scaled)
train_X_with_layer_1.loc[:, pca_cols] = pca_representation
'''

#tsne = TSNE(n_components = 2)
#tsne_representation = tsne.fit_transform(train_X_scaled[knn_features[:15]][:5000])

folds = 5
full_length = len(train_X_scaled)
entries = int(full_length / folds)
full_index = train_X_scaled.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for column in leaky:
    train_X_scaled[column] = 0
for i in range(folds):
    print ('Fold {}'.format(i))
    indices = sorted(random.sample(list(full_index), entries))
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    xtr = train_X_scaled[full_index_throw]
    ytr = train_y[full_index_throw]
    xte = train_X_scaled[full_index_keep]
    
    '''
    knn_features = ['bc_price', 'manager_worths', 'latitude', 'longitude', 'desc_words_length', 'Yday', 'building_worths', 'price_per_bedroom']
    knn_n_closest = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
    knn_columns = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low'] for n_neighbors in knn_n_closest]
    for column in knn_columns:
        train_X_with_layer_1[column] = 0
    for n_neighbors in knn_n_closest:
        #knn_dist = DistanceMetric.get_metric('mahalanobis', V=np.eye(len(knn_n_closest)))
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # metric='manhattan'
        knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
        knn_pred = knn.predict_proba(train_X_scaled[knn_features][full_index_keep])
        new_cols = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = knn_pred
    
    ridge_columns = ['layer1_ridge_predictions']
    for column in ridge_columns:
        train_X_with_layer_1[column] = 0
    ridge = Ridge()
    ridge.fit(xtr, ytr)
    ridge_predictions = ridge.predict(xte)
    train_X_with_layer_1.loc[indices, 'layer1_ridge_predictions'] = ridge_predictions
    
    rf_columns = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low'] for criterion in ['gini', 'entropy']]
    for column in rf_columns:
        train_X_with_layer_1[column] = 0
    for criterion in ['gini', 'entropy']:
        rf = RandomForestClassifier(
            n_estimators=25,
            criterion=criterion,
            max_features='auto',
            min_samples_split=20)

        rf.fit(xtr, ytr)
        rf_predictions = rf.predict_proba(xte)
        new_cols = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = rf_predictions
    
    k_means_columns = ['layer1_k_means_{}_{}'.format(n_clusters, index) for index in range(n_clusters) for n_clusters in [2, 4, 8]]
    for column in k_means_columns:
        train_X_with_layer_1[column] = 0
    for n_clusters in [2, 4, 8]:
        k_means = KMeans(n_clusters=n_clusters)
        k_means.fit(xtr[knn_features], ytr)
        k_means_indices = k_means.predict(xte[knn_features])
        new_cols = ['layer1_k_means_{}_{}'.format(n_clusters, index) for index in range(n_clusters)]
        train_X_with_layer_1.loc[indices, new_cols] = np.eye(n_clusters)[k_means_indices]
    '''

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [180]:
train_X_with_layer_1.to_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')

In [276]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.core import ActivityRegularization
def create_model(xtr, layer1=256, layer2=32, activation='tanh', activation2=PReLU, dropout=0.75):
    model = Sequential()
    
    model.add(Dense(layer1,
                    input_dim=xtr.shape[1],
                    init = 'he_normal',
                    activation=activation))
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(layer2,
                    init = 'he_normal',
                    activation=activation))
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    model.add(BatchNormalization())    
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [268]:
def test_nn_with_columns(X, y, columns_to_include, layer1, layer2, activation, activation2, dropout):
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    ntrain = int(len(X_scaled) * 0.75)

    xtr = X_scaled[columns_to_include][:ntrain]
    ytr = y[:ntrain].values
    ytr_one_hot = np.eye(3)[ytr]
    xte = X_scaled[columns_to_include][ntrain:]
    yte = y[ntrain:].values
    yte_one_hot = np.eye(3)[yte]

    filepath = 'data/nn_weights'

    logger = NBatchLogger()
    early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
    model = create_model(xtr, layer1, layer2, activation, activation2, dropout)
    model.load_weights(filepath)
    return log_loss(yte_one_hot, model.predict(xte.values))

# actually build model

In [274]:
ntrain = int(len(train_X_with_layer_1) * 0.75)

scaler = StandardScaler()

xtr = train_X_with_layer_1[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X_with_layer_1[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [317]:
%%time
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

xgb_cols = all_columns + k_means_columns + knn_columns + ridge_columns + pca_columns + rf_columns

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=xtr[xgb_cols], label=ytr)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

dtest = xgb.DMatrix(data=xte[xgb_cols])
bst = xgb.train(params, dtrain, best_rounds)
xgb_preds = bst.predict(dtest)
print(log_loss(yte_one_hot, xgb_preds))
# 0.539725

[0]	train-mlogloss:1.09111+1.37637e-05	test-mlogloss:1.09132+3.47597e-05
[200]	train-mlogloss:0.572365+0.000966518	test-mlogloss:0.606583+0.00289119
[400]	train-mlogloss:0.495784+0.00111757	test-mlogloss:0.556851+0.00274886
[600]	train-mlogloss:0.46142+0.00115805	test-mlogloss:0.546969+0.00247487
[800]	train-mlogloss:0.435288+0.00122307	test-mlogloss:0.543303+0.00232122
[1000]	train-mlogloss:0.412388+0.00110456	test-mlogloss:0.541574+0.00211891
[1200]	train-mlogloss:0.391678+0.0010726	test-mlogloss:0.540466+0.00189828
[1400]	train-mlogloss:0.372348+0.00112275	test-mlogloss:0.539871+0.00179687
0.5397984
1429
0.546284733463
CPU times: user 3h 44s, sys: 5min 4s, total: 3h 5min 48s
Wall time: 53min 48s


In [307]:
%%time
filepath = 'data/nn_weights'

nn_cols = all_columns + knn_columns + ridge_columns + k_means_columns

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr[nn_cols])
model.fit(xtr[nn_cols].values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte[nn_cols].values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])

model = create_model(xtr[nn_cols])
model.load_weights(filepath)
nn_preds = model.predict(xte[nn_cols].values)
print(log_loss(yte_one_hot, nn_preds))
# 0.58044096863

0.58044096863
CPU times: user 11min 51s, sys: 58.1 s, total: 12min 50s
Wall time: 10min 9s


In [310]:
%%time
et_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[et_cols], ytr)
            
et_train_predictions = test.predict_proba(xtr[et_cols])
et_predictions = test.predict_proba(xte[et_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, et_train_predictions)))
# 0.5989926646715679

0.6084469079023 (test loss 0.5014715811903182)
CPU times: user 10 s, sys: 158 ms, total: 10.2 s
Wall time: 10.2 s


In [309]:
%%time
rf_cols = all_columns# + knn_columns + ridge_columns + k_means_columns

test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[rf_cols], ytr)
            
rf_train_predictions = test.predict_proba(xtr[rf_cols])
rf_predictions = test.predict_proba(xte[rf_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, rf_train_predictions)))
# 0.5814357939419752

0.5867972591837729 (test loss 0.4744615099148531)
CPU times: user 11.2 s, sys: 297 ms, total: 11.5 s
Wall time: 11.6 s


In [304]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[non_sparse_cols], ytr)

knn_train_predictions = test.predict_proba(xtr[non_sparse_cols])
knn_predictions = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6644185840766779 (test loss 0.6542689234451893)
CPU times: user 5min 38s, sys: 4.82 s, total: 5min 43s
Wall time: 5min 55s


In [305]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[knn_features], ytr)

knn_train_predictions = test.predict_proba(xtr[knn_features])
knn_predictions = test.predict_proba(xte[knn_features])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6542388412538083 (test loss 0.6351798773827041)
CPU times: user 49.6 s, sys: 1.34 s, total: 51 s
Wall time: 52.1 s


In [ ]:
print(log_loss(yte_one_hot, xgb_preds))
mixed_preds = (0.75 * xgb_preds) + (0.25 * nn_preds)
print(log_loss(yte_one_hot, mixed_preds))

In [ ]:
train thing on whether or not each classifier did a good job (minimize distance from correct?)
use result to go with the best prediction blended from models

In [45]:
#new_vects = ['tf_idf_' + x[7:] for x in sparse_cols]
#sparse = processed_train[sparse_cols]
#tf_idfs = TfidfTransformer().fit_transform(sparse)
#tf_idfs_df = pd.DataFrame(tf_idfs.toarray(), columns=new_vects)
#processed_train = processed_train.join(tf_idfs_df, how='left')

# Ridge Lasso Elastic

In [308]:
las = Lasso()
enet = ElasticNet()
las.fit(xte, yte)
enet.fit(xte, yte)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [309]:
print (las.score(xte, yte))
predictions = las.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [310]:
print (enet.score(xte, yte))
predictions = enet.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [42]:
for coef, column in zip(enet.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

In [43]:
for coef, column in zip(las.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

# TREES

In [322]:
test = ExtraTreesClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, predictions)))

0.5843451108603969 (test loss 0.3912286072900344)


In [323]:
test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5841850290743527 (test loss 0.3945679429419886)


In [311]:
test = RandomForestClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_predictions), log_loss(ytr_one_hot, _predictions)))

0.5702881670274706 (test loss 0.34260988988913094)


In [312]:
test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
_predictions = test.predict_proba(xtr)
_rf_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, _rf_ent_predictions), log_loss(ytr_one_hot, _predictions)))

0.5690015004443815 (test loss 0.34470505904504534)


In [337]:
pred_mixed = (0.01 * et_predictions + 0.01 * et_ent_predictions + 0.49 * rf_predictions + 0.49 * rf_ent_predictions)
log_loss(yte_one_hot, pred_mixed)

0.56688904952650898

In [12]:
%%time
test = KNeighborsClassifier(n_neighbors=2)
test.fit(xtr[non_sparse_cols], ytr)

predictions = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions = test.predict_proba(xte[non_sparse_cols])

7.076257047822155 (test loss 0.23129250627049086)
CPU times: user 3min 15s, sys: 1.48 s, total: 3min 16s
Wall time: 3min 20s


In [13]:
%%time
test = KNeighborsClassifier(n_neighbors=5)
test.fit(xtr[non_sparse_cols], ytr)

predictions_2 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_2 = test.predict_proba(xte[non_sparse_cols])

2.997161309463761 (test loss 0.4189513500335195)
CPU times: user 3min 36s, sys: 1.02 s, total: 3min 37s
Wall time: 3min 38s


In [14]:
%%time
test = KNeighborsClassifier(n_neighbors=10)
test.fit(xtr[non_sparse_cols], ytr)

predictions_3 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_3 = test.predict_proba(xte[non_sparse_cols])

1.5474359459916005 (test loss 0.4994428601539661)
CPU times: user 3min 48s, sys: 839 ms, total: 3min 49s
Wall time: 3min 50s


In [21]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[non_sparse_cols], ytr)

predictions_6 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_6 = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_6), log_loss(ytr_one_hot, predictions_6)))

0.6344317626130038 (test loss 0.6195293255389913)
CPU times: user 4min 34s, sys: 1.19 s, total: 4min 35s
Wall time: 4min 36s


In [ ]:
for feat, importance in zip(processed_train[cols].columns, clf.feature_importances_):
    if not np.isnan(importance) and importance > 0:
        print (feat, ": ", importance)

In [345]:
print ("nn: ", log_loss(yte_one_hot, nn_predictions))
print ("ab: ", log_loss(yte_one_hot, pred_mixed))
frac_nn = 0.2
averaged_preds = (nn_predictions * frac_nn) + (pred_mixed * (1 - frac_nn))
print ("combo: ", log_loss(yte_one_hot, averaged_preds))

nn:  0.563593691555
ab:  0.566889049527
combo:  0.55827286106


In [ ]:
train_X = pd.read_json('data/train_leaky.json')
train_y = pd.read_json('data/train_interest.json')[0]
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=columns)
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').values
preds = pd.read_csv('data/my_best_preds.csv')

In [295]:
preds = pd.read_csv('data/my_best_preds.csv')

In [313]:
et_clf = ExtraTreesClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='auto',
    min_samples_split=20)
et_clf.fit(train_X, train_y)          
et_predictions = et_clf.predict_proba(test_X)

In [314]:
et_clf_ent = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)
et_clf_ent.fit(train_X, train_y)          
et_ent_predictions = et_clf_ent.predict_proba(test_X)

In [316]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=20)
rf_clf.fit(train_X, train_y)          
rf_predictions = rf_clf.predict_proba(test_X)

In [317]:
rf_clf_ent = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=20)
rf_clf_ent.fit(train_X, train_y)          
rf_ent_predictions = rf_clf_ent.predict_proba(test_X)

In [259]:
nn_predictions = model.predict(test_X.values)

In [297]:
xgb_predictions = preds[['high', 'medium', 'low']].values

In [303]:
new_preds = (0.8 * xgb_predictions) + (0.15 * nn_predictions) + (0.05 * et_predictions)

In [304]:
new_preds_df = pd.DataFrame(new_preds, columns=['high', 'medium', 'low'])
new_preds_df['listing_id'] = preds['listing_id']

In [306]:
new_preds_df.to_csv('data/combined_preds.csv', index=None)